# For using the Roadsign Classifier Web App, go to the following link : https://59bc-34-74-152-238.ngrok-free.app/

If the link changes, just go to the link generated by the forth code block, which is the same code block above the next main header. For running the code on a new Google Colab session, you may need to create a free account with ngrok, uncomment the second code block, and add your authtoken. For updating the code on the server, run all the code blocks

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install streamlit
!pip install pyngrok

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,724 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
# !ngrok authtoken (YOUR NGROK AUTHTOKEN HERE) # MUST UNCOMMENT LINE AND ADD AUTHTOKEN TO RUN FRONT END

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py

### How to Use Model
### 1. Upload .png of roadsign to colab file storage
### 2. change image_path variable to be the name of the roadsign file
# Update the image path to your image
import cv2
import os
import numpy as np
import pytesseract
from google.colab.patches import cv2_imshow
import streamlit as st
from PIL import Image

# We ensure Tesseract is properly set up
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'  # Update if necessary

def detect_road_sign(image):
    original_height, original_width = image.shape[:2]

    # We convert to BGR if the input is in RGB
    image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGR)

    # We resize the image for better processing
    image_resized = cv2.resize(image, (600, 400))

    # We convert to grayscale
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)

    # We apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # We perform edge detection using Canny
    edged = cv2.Canny(blurred, 50, 150)

    # We convert to HSV
    hsv = cv2.cvtColor(image_resized, cv2.COLOR_BGR2HSV)

    # We define color ranges
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Threshold the HSV image for colors
    mask_red = cv2.bitwise_or(cv2.inRange(hsv, lower_red1, upper_red1),
                              cv2.inRange(hsv, lower_red2, upper_red2))
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # We clean up masks with morphological operations
    kernel = np.ones((3, 3), np.uint8)
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_red = cv2.dilate(mask_red, kernel, iterations=1)
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_white = cv2.dilate(mask_white, kernel, iterations=1)
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_yellow = cv2.dilate(mask_yellow, kernel, iterations=1)

    # We detect Stop Signs and No Parking Signs
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours_red:
        area = cv2.contourArea(contour)
        if area > 500:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            perimeter = cv2.arcLength(contour, True)
            circularity = 4 * np.pi * (area / (perimeter * perimeter))

            if circularity > 0.7:  # Circular shapes (for No Parking signs)
                roi = image_resized[y:y + h, x:x + w]
                roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                _, binary_roi = cv2.threshold(roi_gray, 50, 255, cv2.THRESH_BINARY_INV)
                custom_config = r'--psm 6'
                detected_text = pytesseract.image_to_string(binary_roi, config=custom_config)
                if 'P' in detected_text or "NO PARKING" in detected_text.upper():
                    cv2.rectangle(image_resized, (x, y), (x + w, y + h), (0, 255, 0), 3)
                    cv2.putText(image_resized, "No Parking Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                elif len(approx) == 8 and 0.8 < aspect_ratio < 1.2:
                    cv2.drawContours(image_resized, [approx], 0, (0, 0, 255), 3)
                    cv2.putText(image_resized, "Stop Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)



    # We detect Speed Limit Signs
    contours_white, _ = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours_white:
        area = cv2.contourArea(contour)
        if area > 1000:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            if len(approx) == 4 and 0.5 < aspect_ratio < 1.5:
                cv2.drawContours(image_resized, [approx], 0, (255, 0, 0), 3)
                cv2.putText(image_resized, "Speed Limit Sign", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # We detect Yield Signs
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 800:
            epsilon = 0.04 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            if len(approx) == 3:
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = float(w) / h
                if 0.5 < aspect_ratio < 1.5:
                    cv2.drawContours(image_resized, [approx], 0, (0, 255, 255), 3)
                    cv2.putText(image_resized, "Yield Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # We detect Railroad Crossing Signs
    contours_yellow, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = []

    for contour in contours_yellow:
        area = cv2.contourArea(contour)
        if area > 500:  # Lowered the area threshold for detecting smaller or dim regions
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            if 0.7 < aspect_ratio < 1.3:
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * (area / (perimeter * perimeter))
                if circularity > 0.6:
                    bounding_boxes.append((x, y, w, h))

    # We merge bounding boxes into a single region
    if bounding_boxes:
        x_min = min([x for x, y, w, h in bounding_boxes])
        y_min = min([y for x, y, w, h in bounding_boxes])
        x_max = max([x + w for x, y, w, h in bounding_boxes])
        y_max = max([y + h for x, y, w, h in bounding_boxes])

        # We extract the merged bounding box
        roi = image_resized[y_min:y_max, x_min:x_max]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        _, binary_roi = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY_INV)  # Lowered the binary threshold

        # We detect lines or 'X' shape in the ROI
        edges = cv2.Canny(binary_roi, 30, 120)  # Adjusted edge detection thresholds
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=25, minLineLength=20, maxLineGap=10)

        if lines is not None and len(lines) >= 4:  # Expect at least 4 lines for 'X'
            cv2.rectangle(image_resized, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3)
            cv2.putText(image_resized, "Railroad Crossing", (x_min, y_min - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    image_resized = cv2.cvtColor(image_resized, cv2.COLOR_RGBA2BGR)
    image_resized = cv2.resize(image_resized, (original_width, original_height))

    return image_resized

# Streamlit GUI
st.title("Road Sign Detector")
st.write("By Team Charizard")

uploaded_file = st.file_uploader("Choose a road sign image", type=["png", "jpg", "jpeg"])

if uploaded_file is not None:
    # we read the uploaded image
    image = Image.open(uploaded_file)
    image_np = np.array(image)

    # We process the image using the detection function
    processed_image = detect_road_sign(image_np)

    # We display the original and processed images
    col1, col2 = st.columns(2)
    with col1:
        st.image(image, caption="Uploaded Image", use_container_width=True)
    with col2:
        st.image(processed_image, caption="Processed Image", use_container_width=True)



Writing app.py


In [ ]:
from pyngrok import ngrok
!streamlit run app.py &>/content/logs.txt &
public_url = ngrok.connect(8501, "http")

print(f"Streamlit app running at {public_url}")


Streamlit app running at NgrokTunnel: "https://25b7-130-211-250-192.ngrok-free.app" -> "http://localhost:8501"
